# Imports

In [ ]:
import requests

In [ ]:
import sys

In [ ]:
import pyspark.sql.functions as fn

In [ ]:
sys.path.append("../config/")
import config

# Load from Parquet

In [ ]:
df = spark.read.parquet(f'{config.ARTIFACTS}/dataset/M21_cards.parquet')

In [ ]:
df = df.withColumn('keyruneCode', fn.lit('M21'))

# Save to MongoDB 

In [ ]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client['mtggg']
col = db['cards']

col.drop()

In [ ]:
import pymongo_spark
pymongo_spark.activate()

In [ ]:
# as_dict = df.rdd.map(lambda row: row.asDict())
# as_dict.saveToMongoDB(f'mongodb://localhost:27017/mtggg.cards')

# Save to ElasticSearch 

In [ ]:
requests.delete("http://localhost:9200/mtggg")

In [ ]:
requests.put("http://localhost:9200/mtggg", data='{"settings" : {"index" : {"number_of_shards" : 1,"number_of_replicas" : 1}}}')

In [ ]:
requests.post("http://localhost:9200/mtggg/cards/_delete_by_query", data="""{
  "query": {
    "match" : {
        "keyruneCode" : "M21"
    }
  }
}"""
)

In [ ]:
df.write.format("org.elasticsearch.spark.sql")\
    .option("es.resource", "mtggg/cards")\
    .option("es.batch.size.entries", "100")\
    .mode("overwrite")\
    .save()

In [ ]:
!curl 'localhost:9200/mtggg/cards/_search?q=name:acolyte&pretty'